In [24]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))

import pandas as pd
import numpy as np
import re
import json
import importlib

import matplotlib.pyplot as plt
import seaborn as sns

In [69]:
def print_title(title, line_length = 60, symbol = '-'):
    separator = symbol * ((line_length - len(title) - 2) // 2)
    print(f"{separator} {title} {separator}")

# EXPERIMENT DESIGN

## SOURCES INGESTION

-----------------------------------------------------------------------------------------

# EXPERIMENT RESULTS

## SOURCES INGESTION

### LOAD

In [ ]:
# Load raw data
file_name = 'ab_test_example_3.csv'
df_raw = pd.read_csv('../data/' + file_name)
display(df_raw.head(5))

# Make a copy
df = df_raw.copy()

# Check dtypes
print_title('INITIAL DATA TYPES')
print(df.dtypes)

### DTYPES

In [53]:
def format_columns(df, datetime_columns=[], int64_columns=[], float64_columns=[], str_columns=[]):
    for col in datetime_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    for col in int64_columns:
        if col in df.columns:
            df[col] = df[col].astype('Int64')  

    for col in float64_columns:
        if col in df.columns:
            df[col] = df[col].astype('float64')

    for col in str_columns:
        if col in df.columns:
            df[col] = df[col].astype('str')
    
    return df

In [ ]:
datetime_columns = ['WHEN_ENTERED_INTO_EXPERIMENT']
int64_columns = ['U_ID', 'PROMOTER_U_ID', 'IS_TREATMENT', 'COUNT_TRANSFERS', 'DAYS_TRANSACTING', 'DISTINCT_RECEIVERS']
float64_columns = ['TOTAL_TRANSFER_AMOUNT']
str_columns = ['VARIANT', 'USER_SEGMENT', 'PROMO_TAG']

# Basic data conversion
df = format_columns(df, datetime_columns, int64_columns, float64_columns, str_columns)

# Check dtypes
print_title('CONVERTED DATA TYPES')
print(df.dtypes)
display(df.head(5))

## DATA CLEANING

### CHECKS

In [ ]:
variant_column = 'VARIANT'

In [ ]:
# Quick checks on data
print_title('DF INFO')
display(df.info())

print_title('DF DESCRIBE')
display(df.describe())

# Check distribution of variants
print_title('VARIANT DISTRIBUTION')
display(df[variant_column].value_counts())

### DUPLICATES

In [ ]:
# Check for and drop duplicates in the entire DataFrame
duplicated_rows = df.duplicated().sum()
print('# of duplicated rows: ', duplicated_rows)

if duplicated_rows > 0:
    df = df.drop_duplicates()
    print('Duplicates in the DataFrame removed.')
else:
    print('No duplicates in the DataFrame found.')

In [ ]:
primary_key_column = 'U_ID'
timestamp_column = ''

# Check for duplicates in the unique columns
duplicated_rows = df[df[primary_key_column].duplicated(keep=False)]
print(f'# of duplicated on {primary_key_column} column: {duplicated_rows[primary_key_column].nunique()}')

if not duplicated_rows.empty:
    print(f'Duplicated {primary_key_column} and their rows:')
    display(duplicated_rows.sort_values(by = 'U_ID'))

    # Keep only the first following timestamp column order
    if timestamp_column == '':
        df = df.drop_duplicates(subset=primary_key_column, keep='last')
        print('Kept the most recent row for each duplicated U_ID.')
    else:
        df = df.sort_values(timestamp_column).drop_duplicates(subset=primary_key_column, keep='last')
        print('Kept the most recent row for each duplicated U_ID.')

### NULLS

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print_title('NUMBER OF NULL VALUES')
print(missing_values)

In [92]:
# Fill null columns
df['TOTAL_TRANSFER_AMOUNT'] = df['TOTAL_TRANSFER_AMOUNT'].fillna(0)

### INCONSISTENCIES

In [ ]:
# Checks errors in variant labeling
checks = df[(df['IS_TREATMENT'] == 1) & (df['VARIANT'] != 'PRICE_PROMO')]
display(checks)
checks = df[(df['IS_TREATMENT'] == 0) & (df['VARIANT'] != 'CONTROL')]
display(checks)

## METRICS DEFINITION

In [105]:
# calculate the metrics, use metric_cnt_ or metric_cvr_
df['metric_cnt_total_transfer_amount'] = df['TOTAL_TRANSFER_AMOUNT']
df['metric_cvr_transaction'] = df['TOTAL_TRANSFER_AMOUNT'].apply(lambda x: 1 if x > 0 else 0, 1)

## DATA ANALYSIS

In [106]:
# select metrics and columns involved in the test
metric = 'metric_cvr_transaction'
variant_column = 'VARIANT'

### NUMBER OF VARIANTS

In [107]:
num_variants = df[variant_column].nunique()
variant_values = df[variant_column].unique()

print(f"Number of Variants: {num_variants}")
print(f"Variants: {variant_values}")

Number of Variants: 2
Variants: ['CONTROL' 'PRICE_PROMO']


### SAMPLE SIZES

In [108]:
# check if the sample size is large enough
# check if the variant sizes ar equal or not and the proportion

sample_sizes = df[variant_column].value_counts()
print("Sample sizes per variant:")
print(sample_sizes)

variant_proportion = sample_sizes / sample_sizes.sum()
print("\nProportion per variant:")
print(variant_proportion)

variant_ratio = sample_sizes.max() / sample_sizes.min()
print(f"\nVariant Ratio (N = max/min): {variant_ratio:.2f}")

Sample sizes per variant:
VARIANT
CONTROL        29450
PRICE_PROMO    16166
Name: count, dtype: int64

Proportion per variant:
VARIANT
CONTROL        0.645607
PRICE_PROMO    0.354393
Name: count, dtype: float64

Variant Ratio (N = max/min): 1.82


### SAMPLE DISTRIBUTION

In [ ]:
# check if the distribution is normal or not

# add q plots, histogram and violin

### SAMPLE VARIANCES

In [ ]:
# check if the variances are similar between variantes
# add boxplot

## STATISTICAL TESTING

### TEST AND TECHNICHES SELECTION

In [17]:
## Two variants tests
use_mann_whitney_u_test = False # continuous, not normal, works with unbalanced samples
use_welchs_t_test = False # continuous, normal, different variances, works with unbalanced samples (if low unbalance applied although same variants)
use_t_test = False # continuous, normal, equal variances, not recommended if unbalanced
use_z_test = False # continuous data, large sample size, known population real variance, not recommended if unbalanced, not used because you never know the real variance
use_fisher_exact_test = False # proportions, small sample size, works with unbalanced samples, # for proportions, only size matters due to central limit theorem
use_two_proportion_z_test = False # proportions, large sample size, works with unbalanced samples, used alone or post Pearson Chi-square with Bonferroni for multiple variants, # for proportions, only size matters due to central limit theorem

## Multiple variants tests
use_anova_test = False # continuous, normal, not recommended if unbalanced
use_welch_anova_test = False # continuous, normal, if unbalanced
use_pearson_chi_square_test = False # proportions, works with unbalanced samples but needs correction
use_kruskal_wallis_test = False # continuous, not normal, works with unbalanced samples

## Multiple variants post-pairs tests
use_tukey_hsd_test = False # post anova, continuous, not recommended if unbalanced
use_games_howell_test = False # post anova, continuous, for unbalanced samples
use_dunn_test = False # post kruskal wallis, continuous, needs bonferroni, works with unbalanced samples

## Multiple variants correction
use_bonferroni_correction = False # if more than 2 variants and tukey pr games howell is not used

## Unbalance data: N = A/B, if N < 2.5 use normal testing, if N < 5x use balance resampling, if N > 5x use bootstraping
use_balance_resampling = False # for unbalanced sample sizes, equalizing groups, downsampling,
use_bootstraping = False # for small sample sizes or unbalanced groups, estimating confidence intervals, upsampling,

## Additional techniques
use_bayesian_test = False # for probabilistic interpretation, alternative to p-values, small samples
use_permutation_test = False # for distribution-free significance testing, alternative to t-tests or z-tests, extrange distributions

In [20]:
## Segmentation
use_welchs_t_test_segmentation = False # for comparing two segments (A-New vs. B-New), continuous, normal, can unbalanced
use_mann_whitney_u_test_segmentation = False # for comparing two segments (A-New vs. B-New), continuous, not normal, can unbalanced
use_two_proportion_z_test_segmentation = False # for comparing two segments (A-New vs. B-New), proportions, large sample
use_fisher_exact_test_segmentation = False # for comparing two segments (A-New vs. B-New), proportions, small sample

## Interaction Tests, if discrepancies between segments
use_anova_interaction_test_segmentation = False # to test interaction effect between variant and segment, continuous, normal
use_welch_anova_interaction_test_segmentation = False # to test interaction effect between variant and segment, continuous, normal, can unbalanced
use_kruskal_wallis_interaction_test_segmentation = False # to test interaction effect between variant and segment, continuous, not normal
use_logistic_regression_interaction_test_segmentation = False # to test interaction effect for proportions, equivalent to ANOVA for categorical data
# if true, post hoc with tukey, games howell or dunn

### UNBALANCE DATA

#### BALANCE RESAMPLING

#### BOOTSTRAPING

### ADDITIONAL TECHNIQUES

#### BAYESIAN TEST

#### PERMUTATION TEST

## SEGMENT ANALYSIS